In [1]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 4.4 MB/s 


In [2]:
import gurobipy as gp
import numpy as np

In [18]:
#There are 3+2+2+2 decision variables, we are willing to minimize the cost
obj=np.array([50,55,65,5,5,5,7,15,20])

#We have figured out 9 associted constraints on 9 decision variables
A=np.zeros((9,9))
b=np.zeros((9,))
sense=np.array(['<']*9)

#Order- e1,e2,e3,te1,te2,b1,b2,i1,i2

# 1) Horsepower associated
A[0,0:5]=[450,600,750,200,150]
b[0]=600
sense[0]='>'

# 2) Turbo engine and primary engines association
A[1,[0,3]]=[1,-1]
b[1]=0
sense[1]='>'

# 3) Turbo engine and primary engines association
A[2,[1,4]]=[1,-1]
b[2]=0
sense[2]='>'


# 4) Turbo engine and primary engines association
A[3,[2,3,4]]=[1,1,1]
b[3]=1
sense[3]='<'

# 5) Primary engine selection
A[4,[0,1,2]]=[1,1,1]
b[4]=1
sense[4]='='

# 6) Brakes selection
A[5,[5,6]]=[1,1]
b[5]=1
sense[5]='='

# 7) Turbo engines and brakes association 
A[6,[3,4,6]]=[-1,-1,2]
b[6]=0
sense[6]='>'


# 8) Interior decisions
A[7,[7,8]]=[1,1]
b[7]=1
sense[7]='='

# 8) Interior decisions
A[8,[2,3,4,7]]=[1,1,1,-1]
b[8]=0
sense[8]='>'


In [20]:
mod = gp.Model()
mod_x = mod.addMVar(len(obj),vtype=['B']*len(obj)) # 'C' or 'I' or 'B'
mod_con = mod.addMConstrs(A, mod_x, sense, b)
mod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
mod.optimize()

In [21]:
mod_x.x

array([1., 0., 0., 1., 0., 0., 1., 1., 0.])

In [22]:
mod.objVal

77.0

In [16]:
# # A[[0,1],[2,5]]--> (0,2),(1,5)
# # A[np.ix_()]
# A[[1],[2,5]]#--> (0,2),(1,5)#broadcasting of indices
# A[[1,2,3],[2,5]]#--> Will fail to give results

# A[np.ix_([0,1],[2,5])]
# A[np.ix_([0,1,2],[2,5])]

np.concatenate([A,sense.reshape(-1,1),b.reshape(-1,1)],axis=1)

array([['450.0', '600.0', '750.0', '200.0', '150.0', '0.0', '0.0', '0.0',
        '0.0', '>', '600.0'],
       ['1.0', '0.0', '0.0', '-1.0', '0.0', '0.0', '0.0', '0.0', '0.0',
        '>', '0.0'],
       ['0.0', '1.0', '0.0', '0.0', '-1.0', '0.0', '0.0', '0.0', '0.0',
        '>', '0.0'],
       ['0.0', '0.0', '1.0', '1.0', '1.0', '0.0', '0.0', '0.0', '0.0',
        '<', '1.0'],
       ['1.0', '1.0', '1.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
        '=', '1.0'],
       ['0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0', '0.0', '0.0',
        '=', '1.0'],
       ['0.0', '0.0', '0.0', '-1.0', '-1.0', '0.0', '2.0', '0.0', '0.0',
        '>', '0.0'],
       ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0',
        '=', '1.0'],
       ['0.0', '0.0', '1.0', '1.0', '1.0', '0.0', '0.0', '-1.0', '0.0',
        '>', '0.0']], dtype='<U32')